## AKI 예측 모델
- 보호변수 학습에 빼고 할거임

In [150]:
import pandas as pd
train2 = pd.read_csv('checkpoint2_trian.csv')
test2 = pd.read_csv('checkpoint2_test.csv') # 이거 다시 valid test 로 나눠야함
train2.columns

Index(['등록번호', '간호기록 작성일자', 'BT_min', 'BT_max', 'BT_mean', 'DBP_min',
       'DBP_max', 'DBP_mean', 'PR_min', 'PR_max', 'PR_mean', 'SBP_min',
       'SBP_max', 'SBP_mean', 'ALT(GPT)', 'AST(GOT)', 'Albumin',
       'Alkaline Phosphatase', 'Bilirubin, total', 'CO2, total (serum)',
       'Calcium, total', 'Chloride (serum)', 'Phosphorus', 'Potassium(serum)',
       'Sodium(serum)', 'WBC', 'Creatinine', '최초작성일자', '수진일자', '수진 당시 나이',
       '입원일자', '퇴원일자', '입원시 진료과', '기저크레아티닌', '기저eGFR', '중환자실', '입원 일수',
       'ori_target', '성별', 'diagnose', '수진일 당시 직업', '종교', '수진일 당시 결혼상태',
       '음주기간(년)', 'medicine', 'sepsis', 'AKI', 'height', 'weight'],
      dtype='object')

In [151]:
# '최초작성일자','수진일자',   '입원일자', '퇴원일자', '입원 일수', 
# 'ori_target' -> 예측할 AKi 여부는 우리가 직접 정한 기준으로 간다
# 위에 칼럼들 다 버리자 
drop_col = ['최초작성일자','수진일자',   '입원일자', '퇴원일자', '입원 일수', 'ori_target']
train2.drop( columns = drop_col, axis = 1, inplace=True)
test2.drop( columns = drop_col, axis = 1, inplace=True)

슬라이딩 윈도우 적용할라고 일단 환자마다 1일치를 복붙했는데 이거 채워넣어야함

In [153]:
df=train2.copy()
df['간호기록 작성일자'] = pd.to_datetime(df['간호기록 작성일자'])  # 날짜 형식 변환
first_day_data = df.groupby('등록번호').first().reset_index()
first_day_data['간호기록 작성일자'] = first_day_data['간호기록 작성일자'] - pd.Timedelta(days=1)
first_day_data['AKI'] = 0

# 원본 데이터에 추가
df_extended = pd.concat([first_day_data, df], ignore_index=True)

# 결과 정렬
train = df_extended.sort_values(by=['등록번호', '간호기록 작성일자']).reset_index(drop=True)

df=test2.copy()
df['간호기록 작성일자'] = pd.to_datetime(df['간호기록 작성일자'])  # 날짜 형식 변환
first_day_data = df.groupby('등록번호').first().reset_index()
first_day_data['간호기록 작성일자'] = first_day_data['간호기록 작성일자'] - pd.Timedelta(days=1)
first_day_data['AKI'] = 0

# 원본 데이터에 추가
df_extended = pd.concat([first_day_data, df], ignore_index=True)

# 결과 정렬
test = df_extended.sort_values(by=['등록번호', '간호기록 작성일자']).reset_index(drop=True)

In [154]:
train2.columns

Index(['등록번호', '간호기록 작성일자', 'BT_min', 'BT_max', 'BT_mean', 'DBP_min',
       'DBP_max', 'DBP_mean', 'PR_min', 'PR_max', 'PR_mean', 'SBP_min',
       'SBP_max', 'SBP_mean', 'ALT(GPT)', 'AST(GOT)', 'Albumin',
       'Alkaline Phosphatase', 'Bilirubin, total', 'CO2, total (serum)',
       'Calcium, total', 'Chloride (serum)', 'Phosphorus', 'Potassium(serum)',
       'Sodium(serum)', 'WBC', 'Creatinine', '수진 당시 나이', '입원시 진료과', '기저크레아티닌',
       '기저eGFR', '중환자실', '성별', 'diagnose', '수진일 당시 직업', '종교', '수진일 당시 결혼상태',
       '음주기간(년)', 'medicine', 'sepsis', 'AKI', 'height', 'weight'],
      dtype='object')

In [155]:
set(train2.성별)

{0, 1}

In [156]:
# 데이터에 1대1 매핑되는 칼럼들 merge해줄거임)
hadm = ['등록번호']
onebyone = ['기저크레아티닌','중환자실','diagnose', '음주기간(년)','weight', 'height','medicine', 'sepsis','기저eGFR']
hadm.extend(onebyone)
# 보호변수 (성별, 연령, 직업, 종교, 인종, 진료과 등등)
protect_variable = ['성별','종교','수진일 당시 직업','수진 당시 나이','수진일 당시 결혼상태', '입원시 진료과']
hadm.extend(protect_variable)
d_onebyone_table_train = train2[hadm]
d_onebyone_table_train.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)
d_onebyone_table_train

d_onebyone_table_test = test2[hadm]
d_onebyone_table_test.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)
d_onebyone_table_test

/tmp/ipykernel_40820/3494105275.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_onebyone_table_train.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)
/tmp/ipykernel_40820/3494105275.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_onebyone_table_test.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)


,등록번호,기저크레아티닌,중환자실,diagnose,음주기간(년),weight,height,medicine,sepsis,기저eGFR,성별,종교,수진일 당시 직업,수진 당시 나이,수진일 당시 결혼상태,입원시 진료과
0,30,0.97,0.0,0,0.0,56.10,174.0,1,0,76.047496,0,천주교,전문직,75.0,기혼,비뇨의학과
7,142,1.21,1.0,0,0.0,57.00,168.0,1,0,80.415358,0,무교,회사원,29.0,미혼,신경외과
15,150,0.61,0.0,0,0.0,45.59,149.0,1,0,98.533870,1,불교,주부,60.0,기혼,신경외과 (뇌신경센터)
23,195,0.55,0.0,0,0.0,62.20,156.9,1,0,110.138206,1,무교,기타,49.0,기혼,신경외과 (뇌신경센터)
31,322,0.86,0.0,0,0.0,47.45,155.3,1,0,65.618255,1,불교,무직,76.0,사별,종합내과
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24568,229209,0.83,0.0,0,50.0,68.40,171.4,1,0,91.048194,0,무교,자유업,67.0,기혼,응급의학과
24575,229210,0.89,0.0,0,60.0,53.45,163.4,1,0,83.053201,0,무교,무직,76.0,기혼,비뇨의학과
24582,229237,0.78,0.0,0,0.0,67.50,157.2,3,0,75.944082,1,개신교,주부,72.0,기혼,뇌신경센터
24590,229304,1.34,0.0,3,55.0,78.00,175.0,1,0,50.381372,0,무교,기타,78.0,기혼,소화기센터


In [157]:
train2.drop(columns = onebyone,axis =1 , inplace = True)
test2.drop(columns = onebyone,axis =1 , inplace = True)

# 보호변수도 학습에 사용안할거임
train2.drop(columns = protect_variable, axis = 1, inplace = True)
test2.drop(columns = protect_variable, axis = 1, inplace = True)


In [158]:
print(train2.columns, len(train2.columns))
print(test2.columns)

Index(['등록번호', '간호기록 작성일자', 'BT_min', 'BT_max', 'BT_mean', 'DBP_min',
       'DBP_max', 'DBP_mean', 'PR_min', 'PR_max', 'PR_mean', 'SBP_min',
       'SBP_max', 'SBP_mean', 'ALT(GPT)', 'AST(GOT)', 'Albumin',
       'Alkaline Phosphatase', 'Bilirubin, total', 'CO2, total (serum)',
       'Calcium, total', 'Chloride (serum)', 'Phosphorus', 'Potassium(serum)',
       'Sodium(serum)', 'WBC', 'Creatinine', 'AKI'],
      dtype='object') 28
Index(['등록번호', '간호기록 작성일자', 'BT_min', 'BT_max', 'BT_mean', 'DBP_min',
       'DBP_max', 'DBP_mean', 'PR_min', 'PR_max', 'PR_mean', 'SBP_min',
       'SBP_max', 'SBP_mean', 'ALT(GPT)', 'AST(GOT)', 'Albumin',
       'Alkaline Phosphatase', 'Bilirubin, total', 'CO2, total (serum)',
       'Calcium, total', 'Chloride (serum)', 'Phosphorus', 'Potassium(serum)',
       'Sodium(serum)', 'WBC', 'Creatinine', 'AKI'],
      dtype='object')


In [159]:
import pandas as pd
import numpy as np

# 슬라이딩 윈도우 생성 함수
def create_sliding_windows_as_df(data, time_steps=2):
    X, y = [], []
    columns = data.drop(columns=['AKI']).columns
    
    for i in range(len(data) - time_steps):
        # X: 현재 타임스텝에 해당하는 윈도우 데이터 (time_steps * 특성 수 크기)
        window_data = data.iloc[i:i + time_steps].drop(columns=['AKI']).values.flatten()
        X.append(window_data)
        
        # y: 현재 윈도우 끝난 후 48시간 이내의 AKI 여부
        y_values = data.iloc[i + time_steps:i + time_steps + 2]['AKI'].values
        y.append(1 if 1 in y_values else 0)
    
    # 컬럼 이름 만들기 (time_steps별로 컬럼에 번호 붙이기)
    flattened_columns = [f"{col}_t{t}" for t in range(1, time_steps + 1) for col in columns]
    
    # X를 데이터프레임으로 변환
    X_df = pd.DataFrame(X, columns=flattened_columns)
    y_df = pd.Series(y, name='AKI')
    
    return X_df, y_df

# 전체 데이터에 대해 슬라이딩 윈도우 적용
X_all, y_all = [], []
for subject_id, group in train2.groupby('등록번호'):
    group = group.sort_values(by='간호기록 작성일자')
    X, y = create_sliding_windows_as_df(group)
    if not X.empty:  # 비어 있지 않은 경우만 추가
        X_all.append(X)
        y_all.append(y)

# X_all, y_all을 하나의 데이터프레임으로 연결
X_all = pd.concat(X_all, ignore_index=True)  # shape: (샘플 수, time_steps * 특성 수)
y_all = pd.concat(y_all, ignore_index=True)  # shape: (샘플 수,)

# 최종 데이터프레임 형태 확인
print(X_all.shape)  # (샘플 수, time_steps * 특성 수)
print(y_all.shape)  # (샘플 수,)
# X_all과 y_all을 데이터프레임으로 결합
train2_final = pd.concat([X_all, y_all], axis=1)
train2_final.head()  # 최종 데이터프레임 확인
# (25412, 165)
# (25412,)

# 전체 데이터에 대해 슬라이딩 윈도우 적용
X_all, y_all = [], []
for subject_id, group in test2.groupby('등록번호'):
    group = group.sort_values(by='간호기록 작성일자')
    X, y = create_sliding_windows_as_df(group)
    if not X.empty:  # 비어 있지 않은 경우만 추가
        X_all.append(X)
        y_all.append(y)

# X_all, y_all을 하나의 데이터프레임으로 연결
X_all = pd.concat(X_all, ignore_index=True)  # shape: (샘플 수, time_steps * 특성 수)
y_all = pd.concat(y_all, ignore_index=True)  # shape: (샘플 수,)

# 최종 데이터프레임 형태 확인
print(X_all.shape)  # (샘플 수, time_steps * 특성 수)
print(y_all.shape)  # (샘플 수,)
# X_all과 y_all을 데이터프레임으로 결합
test2_final = pd.concat([X_all, y_all], axis=1)
test2_final.head()  # 최종 데이터프레임 확인

(160064, 54)
(160064,)
(17706, 54)
(17706,)


,등록번호_t1,간호기록 작성일자_t1,BT_min_t1,BT_max_t1,BT_mean_t1,DBP_min_t1,DBP_max_t1,DBP_mean_t1,PR_min_t1,PR_max_t1,...,"Bilirubin, total_t2","CO2, total (serum)_t2","Calcium, total_t2",Chloride (serum)_t2,Phosphorus_t2,Potassium(serum)_t2,Sodium(serum)_t2,WBC_t2,Creatinine_t2,AKI
0,30,2022-05-25,36.0,36.6,36.300000,65.0,67.0,66.000000,79.0,87.0,...,1.040,27.0,8.35,105.0,3.05,4.15,140.5,16.34,1.010000,0
1,30,2022-05-26,35.3,36.7,36.163636,49.0,117.0,69.550000,81.0,81.0,...,0.825,27.0,8.05,104.0,2.80,4.50,138.0,9.71,1.133333,0
2,30,2022-05-27,36.3,37.3,36.813333,49.0,64.0,55.600000,92.0,92.0,...,1.210,27.0,7.85,106.0,1.75,4.35,139.5,9.94,0.883333,0
3,30,2022-05-28,36.8,37.3,37.066667,59.0,70.0,63.833333,103.0,103.0,...,1.420,27.0,8.20,103.0,2.30,5.00,139.0,10.30,0.790000,0
4,30,2022-05-29,36.7,36.9,36.800000,62.0,71.0,66.000000,103.0,103.0,...,2.040,27.0,8.90,104.0,2.60,4.40,139.0,8.82,0.770000,0


In [160]:
set(d_onebyone_table_test['성별'])

{0, 1}

In [161]:
train2_final = train2_final[train2_final.columns[~train2_final.columns.str.contains('간호기록 작성일자')]]
train2_final.columns[train2_final.columns.str.contains('등록번호')][1:]
train2_final.drop(columns=list(train2_final.columns[train2_final.columns.str.contains('등록번호')][1:]),
                  axis = 1, inplace=True)
# train2_final
train2_final.rename(columns={'등록번호_t1': '등록번호'}, inplace=True)
train2_final = pd.merge(train2_final,d_onebyone_table_train,how='left',on='등록번호')
train2_final

test2_final = test2_final[test2_final.columns[~test2_final.columns.str.contains('간호기록 작성일자')]]
test2_final.columns[test2_final.columns.str.contains('등록번호')][1:]
test2_final.drop(columns=list(test2_final.columns[test2_final.columns.str.contains('등록번호')][1:]),
                  axis = 1, inplace=True)

test2_final.rename(columns={'등록번호_t1': '등록번호'}, inplace=True)
test2_final = pd.merge(test2_final,d_onebyone_table_test,how='left',on='등록번호')
test2_final

,등록번호,BT_min_t1,BT_max_t1,BT_mean_t1,DBP_min_t1,DBP_max_t1,DBP_mean_t1,PR_min_t1,PR_max_t1,PR_mean_t1,...,height,medicine,sepsis,기저eGFR,성별,종교,수진일 당시 직업,수진 당시 나이,수진일 당시 결혼상태,입원시 진료과
0,30,36.0,36.6,36.300000,65.0,67.0,66.000000,79.0,87.0,84.000000,...,174.0,1,0,76.047496,0,천주교,전문직,75.0,기혼,비뇨의학과
1,30,35.3,36.7,36.163636,49.0,117.0,69.550000,81.0,81.0,81.000000,...,174.0,1,0,76.047496,0,천주교,전문직,75.0,기혼,비뇨의학과
2,30,36.3,37.3,36.813333,49.0,64.0,55.600000,92.0,92.0,92.000000,...,174.0,1,0,76.047496,0,천주교,전문직,75.0,기혼,비뇨의학과
3,30,36.8,37.3,37.066667,59.0,70.0,63.833333,103.0,103.0,103.000000,...,174.0,1,0,76.047496,0,천주교,전문직,75.0,기혼,비뇨의학과
4,30,36.7,36.9,36.800000,62.0,71.0,66.000000,103.0,103.0,103.000000,...,174.0,1,0,76.047496,0,천주교,전문직,75.0,기혼,비뇨의학과
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17701,229356,36.2,36.7,36.533333,73.0,78.0,76.000000,61.0,69.0,64.750000,...,155.0,1,0,101.894396,1,천주교,전문직,56.0,기혼,내분비대사내과
17702,229356,36.6,36.8,36.700000,68.0,76.0,72.666667,69.0,81.0,73.333333,...,155.0,1,0,101.894396,1,천주교,전문직,56.0,기혼,내분비대사내과
17703,229356,36.2,37.4,36.866667,74.0,80.0,76.333333,68.0,75.0,72.333333,...,155.0,1,0,101.894396,1,천주교,전문직,56.0,기혼,내분비대사내과
17704,229356,35.9,37.4,36.226190,46.0,95.0,73.476190,71.0,88.0,78.333333,...,155.0,1,0,101.894396,1,천주교,전문직,56.0,기혼,내분비대사내과


In [162]:
protect_variable

['성별', '종교', '수진일 당시 직업', '수진 당시 나이', '수진일 당시 결혼상태', '입원시 진료과']

In [163]:
# train2_final.drop(columns = protect_variable, axis = 1, inplace = True)
# test2_final.drop(columns = protect_variable, axis = 1, inplace = True)
train2_final.drop(columns =['기저eGFR'], axis = 1, inplace = True)
test2_final.drop(columns = ['기저eGFR'], axis = 1, inplace = True)
# # 보호변수 제거나중에 해야함

In [164]:
train2_final.head()


,등록번호,BT_min_t1,BT_max_t1,BT_mean_t1,DBP_min_t1,DBP_max_t1,DBP_mean_t1,PR_min_t1,PR_max_t1,PR_mean_t1,...,weight,height,medicine,sepsis,성별,종교,수진일 당시 직업,수진 당시 나이,수진일 당시 결혼상태,입원시 진료과
0,9,36.0,37.0,36.400000,35.0,67.0,50.545455,56.0,60.0,57.666667,...,73.7,160.4,1,0,1,불교,무직,77.0,기혼,소화기센터
1,9,36.3,36.6,36.433333,42.0,74.0,56.100000,54.0,58.0,55.750000,...,73.7,160.4,1,0,1,불교,무직,77.0,기혼,소화기센터
2,9,36.3,37.1,36.675000,61.0,73.0,66.400000,54.0,82.0,65.666667,...,73.7,160.4,1,0,1,불교,무직,77.0,기혼,소화기센터
3,9,36.4,36.9,36.700000,66.0,91.0,74.000000,59.0,69.0,63.000000,...,73.7,160.4,1,0,1,불교,무직,77.0,기혼,소화기센터
4,9,36.7,36.9,36.766667,67.0,87.0,74.000000,65.0,83.0,74.000000,...,73.7,160.4,1,0,1,불교,무직,77.0,기혼,소화기센터


In [165]:
train2_final.columns

Index(['등록번호', 'BT_min_t1', 'BT_max_t1', 'BT_mean_t1', 'DBP_min_t1',
       'DBP_max_t1', 'DBP_mean_t1', 'PR_min_t1', 'PR_max_t1', 'PR_mean_t1',
       'SBP_min_t1', 'SBP_max_t1', 'SBP_mean_t1', 'ALT(GPT)_t1', 'AST(GOT)_t1',
       'Albumin_t1', 'Alkaline Phosphatase_t1', 'Bilirubin, total_t1',
       'CO2, total (serum)_t1', 'Calcium, total_t1', 'Chloride (serum)_t1',
       'Phosphorus_t1', 'Potassium(serum)_t1', 'Sodium(serum)_t1', 'WBC_t1',
       'Creatinine_t1', 'BT_min_t2', 'BT_max_t2', 'BT_mean_t2', 'DBP_min_t2',
       'DBP_max_t2', 'DBP_mean_t2', 'PR_min_t2', 'PR_max_t2', 'PR_mean_t2',
       'SBP_min_t2', 'SBP_max_t2', 'SBP_mean_t2', 'ALT(GPT)_t2', 'AST(GOT)_t2',
       'Albumin_t2', 'Alkaline Phosphatase_t2', 'Bilirubin, total_t2',
       'CO2, total (serum)_t2', 'Calcium, total_t2', 'Chloride (serum)_t2',
       'Phosphorus_t2', 'Potassium(serum)_t2', 'Sodium(serum)_t2', 'WBC_t2',
       'Creatinine_t2', 'AKI', '기저크레아티닌', '중환자실', 'diagnose', '음주기간(년)',
       'weight', 'he

### 스케일링전 checkpoint3

In [16]:
# train2_final.to_csv('checkpoint3_train.csv',index = False)
# test2_final.to_csv('checkpoint3_test.csv',index = False)

In [173]:
train2_final = pd.read_csv('checkpoint3_train.csv')
test2_final = pd.read_csv('checkpoint3_test.csv')

In [174]:
train2_final.head(2)

,등록번호,BT_min_t1,BT_max_t1,BT_mean_t1,DBP_min_t1,DBP_max_t1,DBP_mean_t1,PR_min_t1,PR_max_t1,PR_mean_t1,...,weight,height,medicine,sepsis,성별,종교,수진일 당시 직업,수진 당시 나이,수진일 당시 결혼상태,입원시 진료과
0,9,36.0,37.0,36.400000,35.0,67.0,50.545455,56.0,60.0,57.666667,...,73.7,160.4,1,0,1,불교,무직,77.0,기혼,소화기센터
1,9,36.3,36.6,36.433333,42.0,74.0,56.100000,54.0,58.0,55.750000,...,73.7,160.4,1,0,1,불교,무직,77.0,기혼,소화기센터


In [175]:
train2_final.columns

Index(['등록번호', 'BT_min_t1', 'BT_max_t1', 'BT_mean_t1', 'DBP_min_t1',
       'DBP_max_t1', 'DBP_mean_t1', 'PR_min_t1', 'PR_max_t1', 'PR_mean_t1',
       'SBP_min_t1', 'SBP_max_t1', 'SBP_mean_t1', 'ALT(GPT)_t1', 'AST(GOT)_t1',
       'Albumin_t1', 'Alkaline Phosphatase_t1', 'Bilirubin, total_t1',
       'CO2, total (serum)_t1', 'Calcium, total_t1', 'Chloride (serum)_t1',
       'Phosphorus_t1', 'Potassium(serum)_t1', 'Sodium(serum)_t1', 'WBC_t1',
       'Creatinine_t1', 'BT_min_t2', 'BT_max_t2', 'BT_mean_t2', 'DBP_min_t2',
       'DBP_max_t2', 'DBP_mean_t2', 'PR_min_t2', 'PR_max_t2', 'PR_mean_t2',
       'SBP_min_t2', 'SBP_max_t2', 'SBP_mean_t2', 'ALT(GPT)_t2', 'AST(GOT)_t2',
       'Albumin_t2', 'Alkaline Phosphatase_t2', 'Bilirubin, total_t2',
       'CO2, total (serum)_t2', 'Calcium, total_t2', 'Chloride (serum)_t2',
       'Phosphorus_t2', 'Potassium(serum)_t2', 'Sodium(serum)_t2', 'WBC_t2',
       'Creatinine_t2', 'AKI', '기저크레아티닌', '중환자실', 'diagnose', '음주기간(년)',
       'weight', 'he

- standard 스케일링

In [176]:
from sklearn.model_selection import train_test_split

# 고유한 등록번호 추출
unique_ids = train2_final['등록번호'].unique()

# 등록번호 기준으로 80%를 학습, 10%를 검증 데이터셋으로 나누기
train_ids, valid_ids = train_test_split(unique_ids, test_size=0.1, random_state=42)

# 학습 데이터와 검증 데이터 생성
train_df = train2_final[train2_final['등록번호'].isin(train_ids)]
valid_df = train2_final[train2_final['등록번호'].isin(valid_ids)]
test_df = test2_final.copy()
# 결과 확인
print(f"Train Data Size: {len(train_df)}, Valid Data Size: {len(valid_df)}, Test Data Size : {len(test_df)}")

Train Data Size: 144115, Valid Data Size: 15989, Test Data Size : 17706


In [177]:
set(valid_ids).intersection(train_ids)

set()

In [178]:
num_col_to_scale = ['weight', 'height','기저크레아티닌','음주기간(년)'] + [col for col in train2_final.columns if 't1' in col or 't2' in col]
cat_col = ['medicine','diagnose','중환자실','sepsis']
target_col = ['AKI']

from sklearn.preprocessing import StandardScaler

# 스케일러 인스턴스 생성
scaler = StandardScaler()
train_df[num_col_to_scale] = scaler.fit_transform(train_df[num_col_to_scale])

# 테스트 데이터 스케일링
# 주의: 테스트 데이터는 'fit'하지 않고 'transform'만 실행
valid_df[num_col_to_scale] = scaler.transform(valid_df[num_col_to_scale])
test_df[num_col_to_scale] = scaler.transform(test_df[num_col_to_scale])

/tmp/ipykernel_40820/142672034.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[num_col_to_scale] = scaler.fit_transform(train_df[num_col_to_scale])
/tmp/ipykernel_40820/142672034.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df[num_col_to_scale] = scaler.transform(valid_df[num_col_to_scale])


- 범주형 변수 인코딩
    - medicine 칼럼 : {1, 2, 3}로 이루어져 있음 -> 순위가 있는 범주형이라 따로 인코딩 하지 않는다
    - diagnose 칼럼 : 0,1,2,3으로 이루어져 있음 -> 원핫인코딩 : 범주가 순서를 가지는 변수가 아니라서 원핫인코딩 선택
    - 중환자실 칼럼 : {0.0, 1.0} -> 따로 인코딩 x 
    - sepsis : {0,1} -> 따로인코딩 x

In [179]:
# set(train2_final[cat_col[0]]) # {1, 2, 3}
# set(train2_final[cat_col[1]]) # {0, 1, 2, 3}
# set(train2_final[cat_col[2]]) # {0.0, 1.0}
set(train2_final[cat_col[3]]) #  {0, 1}

{0, 1}

In [180]:
train_df['중환자실'] = train_df['중환자실'].astype(int)
valid_df['중환자실'] = valid_df['중환자실'].astype(int)
test_df['중환자실'] = test_df['중환자실'].astype(int)

/tmp/ipykernel_40820/621416000.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['중환자실'] = train_df['중환자실'].astype(int)
/tmp/ipykernel_40820/621416000.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df['중환자실'] = valid_df['중환자실'].astype(int)


In [181]:
# One-Hot Encoding for 'diagnose' (순서가 없는 다중 범주형 변수)
train_df = pd.get_dummies(train_df, columns=['diagnose'], prefix='diag')
valid_df = pd.get_dummies(valid_df, columns=['diagnose'], prefix='diag')
test_df = pd.get_dummies(test_df, columns=['diagnose'], prefix='diag')

# train, valid, test 데이터셋 간의 칼럼 수를 맞추기 위해 align 사용
train_df, valid_df = train_df.align(valid_df, join='outer', axis=1, fill_value=0)
train_df, test_df = train_df.align(test_df, join='outer', axis=1, fill_value=0)
valid_df, test_df = valid_df.align(test_df, join='outer', axis=1, fill_value=0)

# 인코딩된 'diag' 칼럼들을 정수형(int)으로 변환
diag_columns = [col for col in train_df.columns if col.startswith('diag_')]
train_df[diag_columns] = train_df[diag_columns].astype(int)
valid_df[diag_columns] = valid_df[diag_columns].astype(int)
test_df[diag_columns] = test_df[diag_columns].astype(int)

### checkpoint4

In [182]:
train_df.to_csv('checkpoint4_train.csv',index = False)
test_df.to_csv('checkpoint4_test.csv',index = False)
valid_df.to_csv('checkpoint4_valid.csv',index = False)
# checkpoint4_tabnet_train.csv들이랑 수치형 스케일링 방식이 다름

In [86]:
import pandas as pd
train_df = pd.read_csv('checkpoint4_train.csv')
test_df = pd.read_csv('checkpoint4_train.csv')
valid_df = pd.read_csv('checkpoint4_train.csv')

In [87]:
train_df.head()

,등록번호,BT_min_t1,BT_max_t1,BT_mean_t1,DBP_min_t1,DBP_max_t1,DBP_mean_t1,PR_min_t1,PR_max_t1,PR_mean_t1,...,성별,종교,수진일 당시 직업,수진 당시 나이,수진일 당시 결혼상태,입원시 진료과,diag_0,diag_1,diag_2,diag_3
0,9,-0.299301,-0.606611,-0.771342,-1.958383,-1.065636,-1.999416,-1.111034,-1.599408,-1.600382,...,1,불교,무직,77.0,기혼,소화기센터,1,0,0,0
1,9,-0.038846,-1.165525,-0.718236,-1.400487,-0.646573,-1.457801,-1.234416,-1.686045,-1.715365,...,1,불교,무직,77.0,기혼,소화기센터,1,0,0,0
2,9,-0.038846,-0.466882,-0.333215,0.113800,-0.706439,-0.453463,-1.234416,-0.646393,-1.120453,...,1,불교,무직,77.0,기혼,소화기센터,1,0,0,0
3,9,0.047972,-0.746339,-0.293385,0.512297,0.371151,0.287601,-0.925962,-1.209538,-1.280430,...,1,불교,무직,77.0,기혼,소화기센터,1,0,0,0
4,9,0.308427,-0.746339,-0.187172,0.591996,0.131686,0.287601,-0.555818,-0.603074,-0.620527,...,1,불교,무직,77.0,기혼,소화기센터,1,0,0,0


- 전처리 끝났고 진짜 모델

In [88]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric
from aif360.algorithms.preprocessing import Reweighing
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score


In [57]:
import os
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.linear_model import LogisticRegression,RidgeClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance

import lightgbm as lgb
from lightgbm import LGBMClassifier
from lightgbm import plot_importance

from sklearn.model_selection import train_test_split as tts
# from bayes_opt import BayesianOptimization
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import log_loss

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score
import random
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)

import os
import warnings
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt

os.environ['KMP_DUPLICATE_LIB_OK']='True'
warnings.filterwarnings("ignore", category=FutureWarning)

In [89]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[0, 1])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[0, 1])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[0, 1])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도(precision 0.2 목표): {:.4f}".format(precision))
    print("재현율(recall 0.75 목표): {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [133]:
# print(hadm)
# hadm.remove('height')
# hadm.remove('weight')
# hadm.remove('diagnose')
# hadm.remove('medicine')
# print(hadm)

In [90]:
protect_variable = ['성별', '종교', '수진일 당시 직업', '수진 당시 나이', '수진일 당시 결혼상태', '입원시 진료과']
등록번호_보호변수 = ['등록번호'] + protect_variable
등록번호_보호변수

['등록번호', '성별', '종교', '수진일 당시 직업', '수진 당시 나이', '수진일 당시 결혼상태', '입원시 진료과']

In [28]:
# from sklearn.preprocessing import LabelEncoder
# # 보호변수에 대해서 라벨인코딩
# # 각 범주형 변수를 라벨 인코딩
# label_encoders = {}  # 각 변수를 위한 라벨 인코더 저장용
# for col in protect_variable:
#     le = LabelEncoder()
#     train_df[col] = le.fit_transform(train_df[col])
#     valid_df[col] = le.transform(valid_df[col])
#     test_df[col] = le.transform(test_df[col])
#     label_encoders[col] = le  # 나중에 필요할 수 있어 인코더 저장

# OneHotEncoder를 사용하여 원핫인코딩
onehot_enc = OneHotEncoder(sparse=False)
# 원핫 인코딩을 보호변수에 적용
X_train_protect = onehot_enc.fit_transform(train_df[protect_variable])
X_valid_protect = onehot_enc.transform(valid_df[protect_variable])
X_test_protect = onehot_enc.transform(test_df[protect_variable])

# 원핫 인코딩된 보호변수 열 이름 생성
encoded_protect_vars = onehot_enc.get_feature_names_out(protect_variable)

# 인코딩된 보호변수 추가 후 데이터 병합
X_train_full = pd.concat([pd.DataFrame(X_train_protect, columns=encoded_protect_vars), 
                          train_df.drop(protect_variable + ['등록번호', 'AKI'], axis=1)], axis=1)
X_valid_full = pd.concat([pd.DataFrame(X_valid_protect, columns=encoded_protect_vars), 
                          valid_df.drop(protect_variable + ['등록번호', 'AKI'], axis=1)], axis=1)
X_test_full = pd.concat([pd.DataFrame(X_test_protect, columns=encoded_protect_vars), 
                         test_df.drop(protect_variable + ['등록번호', 'AKI'], axis=1)], axis=1)


# BinaryLabelDataset으로 변환
train_dataset = BinaryLabelDataset(df=pd.concat([X_train_full, y_train], axis=1), 
                                   label_names=['AKI'], 
                                   protected_attribute_names=encoded_protect_vars.tolist())

valid_dataset = BinaryLabelDataset(df=pd.concat([X_valid_full, y_valid], axis=1), 
                                   label_names=['AKI'], 
                                   protected_attribute_names=encoded_protect_vars.tolist())

test_dataset = BinaryLabelDataset(df=pd.concat([X_test_full, y_test], axis=1), 
                                  label_names=['AKI'], 
                                  protected_attribute_names=encoded_protect_vars.tolist())


In [31]:
# 1. 보호 변수를 제거하지 않고 전체 데이터셋을 유지 (등록번호는 제외) -> 이거 공정성 개선 전처리떄 쓸거임
X_train_full = train_df[train_df.columns.drop(['AKI','등록번호'])]
X_valid_full = valid_df[valid_df.columns.drop(['AKI','등록번호'])]
X_test_full = test_df[test_df.columns.drop(['AKI','등록번호'])]


In [32]:
X_train = train_df[train_df.columns.drop('AKI')].drop(columns =등록번호_보호변수,axis = 1)
y_train = train_df['AKI']
X_valid = valid_df[valid_df.columns.drop('AKI')].drop(columns = 등록번호_보호변수,axis = 1)
y_valid = valid_df['AKI']
X_test = test_df[test_df.columns.drop('AKI')].drop(columns =등록번호_보호변수,axis = 1)
y_test = test_df['AKI']

In [33]:
# 양성 클래스와 음성 클래스의 비율 계산
ratio =(len(y_train) - sum(y_train)) / sum(y_train)
ratio

8.57193145589798

In [36]:
# AIF360 BinaryLabelDataset으로 변환 (학습 데이터 기준, 검증 및 테스트 데이터도 동일한 방식 적용 가능)
train_dataset = BinaryLabelDataset(df=pd.concat([X_train_full, y_train], axis=1), 
                                   label_names=['AKI'], 
                                   protected_attribute_names=protect_variable)

valid_dataset = BinaryLabelDataset(df=pd.concat([X_valid_full, y_valid], axis=1), 
                                   label_names=['AKI'], 
                                   protected_attribute_names=protect_variable)

test_dataset = BinaryLabelDataset(df=pd.concat([X_test_full, y_test], axis=1), 
                                  label_names=['AKI'], 
                                  protected_attribute_names=protect_variable)

In [39]:
# BinaryLabelDataset에서 피처와 라벨 추출
X_train_full = train_dataset.features
X_valid_full = valid_dataset.features
X_test_full = test_dataset.features

y_train = train_dataset.labels.ravel()  # 라벨을 1차원 배열로 변환
y_valid = valid_dataset.labels.ravel()
y_test = test_dataset.labels.ravel()


In [48]:
#  보호 변수를 제외한 학습용 데이터 생성
# train_dataset.feature_names에는 모든 피처 이름이 있음
feature_names = train_dataset.feature_names
print(feature_names)

# 보호 변수를 제외한 피처 인덱스 구하기
exclude_indices = [feature_names.index(var) for var in protect_variable]
include_indices = [i for i in range(len(feature_names)) if i not in exclude_indices]
print(exclude_indices)
print(include_indices)

# 보호 변수를 제외한 X_train, X_valid, X_test 생성
X_train = X_train_full[:, include_indices]
X_valid = X_valid_full[:, include_indices]
X_test = X_test_full[:, include_indices]

#  확인
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_valid shape: {X_valid.shape}")
print(f"y_valid shape: {y_valid.shape}")

['BT_min_t1', 'BT_max_t1', 'BT_mean_t1', 'DBP_min_t1', 'DBP_max_t1', 'DBP_mean_t1', 'PR_min_t1', 'PR_max_t1', 'PR_mean_t1', 'SBP_min_t1', 'SBP_max_t1', 'SBP_mean_t1', 'ALT(GPT)_t1', 'AST(GOT)_t1', 'Albumin_t1', 'Alkaline Phosphatase_t1', 'Bilirubin, total_t1', 'CO2, total (serum)_t1', 'Calcium, total_t1', 'Chloride (serum)_t1', 'Phosphorus_t1', 'Potassium(serum)_t1', 'Sodium(serum)_t1', 'WBC_t1', 'Creatinine_t1', 'BT_min_t2', 'BT_max_t2', 'BT_mean_t2', 'DBP_min_t2', 'DBP_max_t2', 'DBP_mean_t2', 'PR_min_t2', 'PR_max_t2', 'PR_mean_t2', 'SBP_min_t2', 'SBP_max_t2', 'SBP_mean_t2', 'ALT(GPT)_t2', 'AST(GOT)_t2', 'Albumin_t2', 'Alkaline Phosphatase_t2', 'Bilirubin, total_t2', 'CO2, total (serum)_t2', 'Calcium, total_t2', 'Chloride (serum)_t2', 'Phosphorus_t2', 'Potassium(serum)_t2', 'Sodium(serum)_t2', 'WBC_t2', 'Creatinine_t2', '기저크레아티닌', '중환자실', '음주기간(년)', 'weight', 'height', 'medicine', 'sepsis', '성별', '종교', '수진일 당시 직업', '수진 당시 나이', '수진일 당시 결혼상태', '입원시 진료과', 'diag_0', 'diag_1', 'diag_2', 'd

## xgb & 공정성까지

### xgb

- 정확도: 0.8475
- 정밀도(precision 0.2 목표): 0.3907
- 재현율(recall 0.75 목표): 0.8338
- F1: 0.5321

In [58]:
import optuna
# from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# Optuna를 위한 목적 함수
def objective(trial):
    # 탐색할 하이퍼파라미터 설정
    param = {
        'random_state': 42,
        'scale_pos_weight': trial.suggest_loguniform('scale_pos_weight', ratio-2, ratio+2),
        'n_estimators': trial.suggest_int('n_estimators', 300, 500),
        'alpha': trial.suggest_loguniform('alpha', 1e-5, 1e-1),
        'gamma': trial.suggest_loguniform('gamma', 1e-6, 1e-1),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 12)
    }

    # 모델 생성 및 학습
    model = xgb.XGBClassifier(**param)
    model.fit(X_train, y_train)

    # 예측 및 평가
    y_pred = model.predict(X_valid)
#     score = f1_score(y_test, y_pred, average='binary')
    # score = precision_score(y_test, y_pred, average='binary')
    score = recall_score(y_valid, y_pred, average='binary')
    return score

# Optuna Study 생성 및 최적화
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=60)

# 최적화 결과 출력
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2024-10-18 03:34:13,646] A new study created in memory with name: no-name-2cc8d6d6-24f7-457b-a29a-92e61320695e
[W 2024-10-18 03:34:17,395] Trial 0 failed with parameters: {'scale_pos_weight': 10.482069099598254, 'n_estimators': 452, 'alpha': 0.005650739023269271, 'gamma': 1.6007716692849576e-05, 'learning_rate': 0.20520709234595763, 'max_depth': 12} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\anaconda3\envs\myenv\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\김나현\AppData\Local\Temp\ipykernel_35584\3083523624.py", line 20, in objective
    model.fit(X_train, y_train)
  File "c:\anaconda3\envs\myenv\Lib\site-packages\xgboost\core.py", line 726, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "c:\anaconda3\envs\myenv\Lib\site-packages\xgboost\sklearn.py", line 1531, in fit
    self._Booster = train(
    

KeyboardInterrupt: 

In [59]:
recall_81 = {'scale_pos_weight': 10.058931188509192, 'n_estimators': 448, 'alpha': 0.0002947216437764358, 'gamma': 0.00022202473060981608, 'learning_rate': 0.020427139052839147, 'max_depth': 4}

In [60]:
model_xgb_pre = xgb.XGBClassifier(random_state=42,
                              # scale_pos_weight=ratio,
                              scale_pos_weight=recall_81['scale_pos_weight'],
                              max_depth = recall_81['max_depth'],
                              n_estimators= recall_81['n_estimators'],
                              alpha = recall_81['alpha'],
                              gamma = recall_81['gamma'],
                              learning_rate = recall_81['learning_rate'],
                             ) 

model_xgb_pre.fit(X_train, y_train)


XGBClassifier(alpha=0.0002947216437764358, base_score=None, booster=None,
              callbacks=None, colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.00022202473060981608, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.020427139052839147, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=448, n_jobs=None,
              num_parallel_tree=None, ...)

In [61]:
pred_xgb = model_xgb_pre.predict(X_valid)
get_clf_eval(y_valid, pred_xgb)
# 정밀도 0.2 목표
# 재현율 0.75 목표

오차행렬:
 [[109729  19330]
 [  2397  12659]]

정확도: 0.8492
정밀도(precision 0.2 목표): 0.3957
재현율(recall 0.75 목표): 0.8408
F1: 0.5382


In [82]:
# 최적의 하이퍼파라미터를 사용하여 모델 학습
model_xgb_pre = xgb.XGBClassifier(random_state=42,
                              # scale_pos_weight=ratio,
                              scale_pos_weight=recall_81['scale_pos_weight'],
                              max_depth = recall_81['max_depth'],
                              n_estimators= recall_81['n_estimators'],
                              alpha = recall_81['alpha'],
                              gamma = recall_81['gamma'],
                              learning_rate = recall_81['learning_rate'])

import pandas as pd

# 보호변수 제외한 feature들
a_sub_b = [x for x in train_dataset.feature_names if x not in  protect_variable]
# X_train, X_valid, y_train, y_valid를 DataFrame으로 변환
X_train_concat = pd.DataFrame(X_train, columns=a_sub_b)
X_valid_concat = pd.DataFrame(X_valid, columns=a_sub_b)

y_train_concat = pd.DataFrame(y_train, columns=['AKI'])
y_valid_concat = pd.DataFrame(y_valid, columns=['AKI'])

# pandas DataFrame으로 변환한 후 concat 사용
X_train_full_concat = pd.concat([X_train_concat, X_valid_concat], ignore_index=True)
y_train_full_concat = pd.concat([y_train_concat, y_valid_concat], ignore_index=True)
y_train_full_concat = y_train_full_concat.values.ravel()  # 또는 squeeze()

# 모델 학습
model_xgb_pre.fit(X_train_full_concat, y_train_full_concat)

# 테스트 데이터셋으로 모델 성능 평가
y_pred = model_xgb_pre.predict(X_test)
get_clf_eval(y_test, y_pred)


오차행렬:
 [[109816  19243]
 [  2408  12648]]

정확도: 0.8498
정밀도(precision 0.2 목표): 0.3966
재현율(recall 0.75 목표): 0.8401
F1: 0.5388


In [70]:
print(a_sub_b)

['BT_min_t1', 'BT_max_t1', 'BT_mean_t1', 'DBP_min_t1', 'DBP_max_t1', 'DBP_mean_t1', 'PR_min_t1', 'PR_max_t1', 'PR_mean_t1', 'SBP_min_t1', 'SBP_max_t1', 'SBP_mean_t1', 'ALT(GPT)_t1', 'AST(GOT)_t1', 'Albumin_t1', 'Alkaline Phosphatase_t1', 'Bilirubin, total_t1', 'CO2, total (serum)_t1', 'Calcium, total_t1', 'Chloride (serum)_t1', 'Phosphorus_t1', 'Potassium(serum)_t1', 'Sodium(serum)_t1', 'WBC_t1', 'Creatinine_t1', 'BT_min_t2', 'BT_max_t2', 'BT_mean_t2', 'DBP_min_t2', 'DBP_max_t2', 'DBP_mean_t2', 'PR_min_t2', 'PR_max_t2', 'PR_mean_t2', 'SBP_min_t2', 'SBP_max_t2', 'SBP_mean_t2', 'ALT(GPT)_t2', 'AST(GOT)_t2', 'Albumin_t2', 'Alkaline Phosphatase_t2', 'Bilirubin, total_t2', 'CO2, total (serum)_t2', 'Calcium, total_t2', 'Chloride (serum)_t2', 'Phosphorus_t2', 'Potassium(serum)_t2', 'Sodium(serum)_t2', 'WBC_t2', 'Creatinine_t2', '기저크레아티닌', '중환자실', '음주기간(년)', 'weight', 'height', 'medicine', 'sepsis', 'diag_0', 'diag_1', 'diag_2', 'diag_3']


In [77]:
# X_valid_full과 y_valid를 DataFrame으로 변환
X_valid_df = pd.DataFrame(X_valid_full, columns=train_dataset.feature_names)
y_valid_df = pd.DataFrame(y_valid, columns=['AKI'])

# X_valid_df와 y_valid_df를 concat하여 BinaryLabelDataset 생성
valid_dataset_pred = BinaryLabelDataset(df=pd.concat([X_valid_df, y_valid_df], axis=1).assign(AKI=pred_xgb),
                                        label_names=['AKI'],
                                        protected_attribute_names=protect_variable)

# 공정성 지표 계산
metric = ClassificationMetric(valid_dataset, valid_dataset_pred,
                              unprivileged_groups=[{'성별': 0}],
                              privileged_groups=[{'성별': 1}])

eod = metric.equal_opportunity_difference()
di = metric.disparate_impact()

print(f"Equal Opportunity Difference: {eod}")
print(f"Disparate Impact: {di}")

Equal Opportunity Difference: 0.007292927274929251
Disparate Impact: 1.3036628858779074


### 공정성 전처리 적용

In [83]:
from aif360.algorithms.preprocessing import Reweighing

# unprivileged_groups와 privileged_groups 설정 (예: 성별 기준)
unprivileged_groups = [{'성별': 1}]  # 비특권 그룹 (예: 여성)
privileged_groups = [{'성별': 0}]    # 특권 그룹 (예: 남성)

# Reweighing 전처리 적용
RW = Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
train_dataset_rw = RW.fit_transform(train_dataset)

# 학습용 데이터에서 보호 변수를 제외한 피처 추출
X_train_rw = train_dataset_rw.features[:, include_indices]
y_train_rw = train_dataset_rw.labels.ravel()


## 모델 재학습

In [84]:
from xgboost import XGBClassifier

# Reweighing으로 가중치 계산
sample_weights = train_dataset_rw.instance_weights

# XGBoost 모델 초기화 및 학습 (가중치 반영)
model_xgb_rw = XGBClassifier(random_state=42,
                             scale_pos_weight=recall_81['scale_pos_weight'],
                             max_depth=recall_81['max_depth'],
                             n_estimators=recall_81['n_estimators'],
                             alpha=recall_81['alpha'],
                             gamma=recall_81['gamma'],
                             learning_rate=recall_81['learning_rate'])

model_xgb_rw.fit(X_train_rw, y_train_rw, sample_weight=sample_weights)


XGBClassifier(alpha=0.0002947216437764358, base_score=None, booster=None,
              callbacks=None, colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.00022202473060981608, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.020427139052839147, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=448, n_jobs=None,
              num_parallel_tree=None, ...)

# 공정성 지표 재계산 및 비교

- 공정성 지표 해석
    - Disparate Impact (DI): 1에 가까울수록 공정합니다. 1 미만일 경우 비특권 그룹에 불리한 예측을 하고 있음을 의미합니다.
    - Equal Opportunity Difference (EOD): 0에 가까울수록 공정합니다. 양수나 음수가 클수록 특정 그룹에 대한 재현율의 차이가 큼을 나타냅니다.

In [85]:
# 검증 데이터에서 예측 수행
y_pred_rw = model_xgb_rw.predict(X_valid)

# 성능 평가
get_clf_eval(y_valid, y_pred_rw)

# 공정성 지표 계산
valid_dataset_pred_rw = valid_dataset.copy(deepcopy=True)
valid_dataset_pred_rw.labels = y_pred_rw.reshape(-1, 1)

metric_rw = ClassificationMetric(valid_dataset, valid_dataset_pred_rw,
                                 unprivileged_groups=unprivileged_groups,
                                 privileged_groups=privileged_groups)

eod_rw = metric_rw.equal_opportunity_difference()
di_rw = metric_rw.disparate_impact()

print(f"Reweighed Equal Opportunity Difference: {eod_rw}")
print(f"Reweighed Disparate Impact: {di_rw}")


오차행렬:
 [[109769  19290]
 [  2386  12670]]

정확도: 0.8496
정밀도(precision 0.2 목표): 0.3964
재현율(recall 0.75 목표): 0.8415
F1: 0.5390
Reweighed Equal Opportunity Difference: 0.008581344426833337
Reweighed Disparate Impact: 0.8137861118647307


다중 보호 변수 고려: 현재 예시에서는 성별을 기준으로 공정성을 평가했습니다. 다중 보호 변수를 고려하려면 unprivileged_groups와 privileged_groups를 복합적으로 정의해야 합니다.

python
코드 복사
unprivileged_groups = [
    {'성별': 0, '종교': 0},
    {'성별': 0, '수진일 당시 직업': 0},
    # 추가 그룹 정의
]

privileged_groups = [
    {'성별': 1, '종교': 1},
    {'성별': 1, '수진일 당시 직업': 1},
    # 추가 그룹 정의
]
다른 전처리 기법 활용: Reweighing 외에도 DisparateImpactRemover, OptimizedPreprocessing, LearningFairRepresentation 등의 전처리 기법을 시도해 볼 수 있습니다.

모델 후처리 및 교정: 전처리 외에도 모델 후처리 단계에서 공정성을 개선하는 방법들이 있습니다. 예를 들어, Reject Option Classification을 적용할 수 있습니다.

하이퍼파라미터 최적화: Optuna와 같은 라이브러리를 활용해 모델의 하이퍼파라미터를 최적화하면서 공정성과 성능을 동시에 고려할 수 있습니다.

결론
위의 과정을 통해 AIF360을 활용하여 공정성을 평가하고 개선할 수 있습니다. 공정성 전처리를 적용한 후 모델을 재학습하고, 이전 모델과의 공정성 지표를 비교함으로써 공정성 향상 여부를 객관적으로 평가할 수 있습니다. 공정성을 고려한 모델 구축은 특히 민감한 데이터를 다루는 의료 분야에서 매우 중요하므로, 이러한 과정을 체계적으로 수행하는 것이 필수적입니다.

궁금한 점이나 추가적인 도움이 필요하시면 언제든지 문의해 주세요!

### 공정성

In [52]:
# print(f'train최소 나이 : {min(set(train_df['수진 당시 나이']))} \ntrain 최대 나이 : {max(set(train_df['수진 당시 나이']))}\n')
# print(f'test최소 나이 : {min(set(test_df['수진 당시 나이']))} \ntest 최대 나이 : {max(set(test_df['수진 당시 나이']))}\n')
# print(f'valid최소 나이 : {min(set(valid_df['수진 당시 나이']))} \nvalid 최대 나이 : {max(set(valid_df['수진 당시 나이']))}\n')


- 만나이라서 19세 -> 20대 취급해서 공정성 계산 하겠음
- 이런식으로 나이는 x9세는 한살 더해서 나이대 어디에 해당하는지 반영하겠음.

In [44]:
print(protect_variable)
for i in protect_variable:
    print('\n')
    print(test_df[i].value_counts().sort_index())
    print('\n')
# age_group_counts = test_df['age_group_big'].value_counts().sort_index()
# print(f'\n\n{age_group_counts}')

['성별', '종교', '수진일 당시 직업', '수진 당시 나이', '수진일 당시 결혼상태', '입원시 진료과']


성별
0    10201
1     7505
Name: count, dtype: int64




종교
0       243
개신교    4575
기타      294
무교     7655
불교     3013
천주교    1926
Name: count, dtype: int64




수진일 당시 직업
0       275
군인       92
기타     1479
무직     7451
자유업    1800
전문직     848
주부     3652
학생      194
회사원    1915
Name: count, dtype: int64




수진 당시 나이
19.0     42
20.0     55
21.0     38
22.0     65
23.0     34
       ... 
86.0    308
87.0    249
88.0    257
89.0    188
90.0    584
Name: count, Length: 72, dtype: int64




수진일 당시 결혼상태
0       266
기혼    13322
동거       64
미혼     1195
별거       91
사별     2105
이혼      663
Name: count, dtype: int64




입원시 진료과
감염내과                  251
관절센터                  321
내과                      6
내분비대사내과                61
내분비대사내과 (노인의료센터)        6
노인병내과 (노인의료센터)        267
노인의료센터                 62
뇌신경센터                 349
류마티스내과                 39
마취통증의학과                31
비뇨의학과                 630
비뇨의학과 (암센터)            

In [71]:
## 나이 그룹 관련 함수들 *****************************
def 만나이플러스1(df):
    data = df
    data['수진 당시 나이+1(만나이x)'] = data['수진 당시 나이']+1
    return data
    
# 20대 30대 이렇게 구별해주는 함수 
def map_age_to_decade(age):
    return f"{(age // 10) * 10}대"
    
def classify_age_group(age):
    if age <= 29:
        return 'Young Adults'  # 청년층
    elif 30 <= age <= 64:
        return 'Middle-aged Adults'  # 중장년층
    elif age >= 65:
        return 'Older Adults'  # 노년층
    return 'Unknown'  # 예외 처리

# 입원시 진료과 그룹 줄이는 함수
# 데이터프레임이 입력이고 biggest,bigger 로 그룹화한 칼럼 붙여서 데이터 프레임 반환
def classify_department_biggest(df):
    biggest_mapping = {
    '감염내과': '기타 진료과',
    '관절센터': '전문 센터 계열',
    '내과': '내과 계열',
    '내분비대사내과': '내과 계열',
    '내분비대사내과 (노인의료센터)': '내과 계열',
    '노인병내과 (노인의료센터)': '내과 계열',
    '노인의료센터': '전문 센터 계열',
    '뇌신경센터': '전문 센터 계열',
    '류마티스내과': '내과 계열',
    '마취통증의학과': '기타 진료과',
    '비뇨의학과': '외과 계열',
    '비뇨의학과 (암센터)': '외과 계열',
    '산부인과': '외과 계열',
    '성형외과': '외과 계열',
    '소아청소년과 (알레르기호흡기)': '소아·청소년과 및 재활 계열',
    '소화기내과': '내과 계열',
    '소화기내과 (암센터)': '내과 계열',
    '소화기센터': '전문 센터 계열',
    '순환기내과 (심장혈관센터)': '내과 계열',
    '신경과 (뇌신경센터)': '내과 계열',
    '신경외과': '외과 계열',
    '신경외과 (뇌신경센터)': '외과 계열',
    '신경외과 (척추센터)': '외과 계열',
    '신장내과': '내과 계열',
    '심장혈관센터': '전문 센터 계열',
    '심장혈관흉부외과 (심장혈관센터)': '외과 계열',
    '심장혈관흉부외과 (폐센터)': '외과 계열',
    '안과': '기타 진료과',
    '알레르기내과': '내과 계열',
    '외과': '외과 계열',
    '외과 (암센터)': '외과 계열',
    '응급의학과': '기타 진료과',
    '이비인후과': '기타 진료과',
    '재활의학과': '소아·청소년과 및 재활 계열',
    '정형외과 (관절센터)': '외과 계열',
    '정형외과 (암센터)': '외과 계열',
    '정형외과 (척추센터)': '외과 계열',
    '종합내과': '내과 계열',
    '척추센터': '전문 센터 계열',
    '치과': '외과 계열',
    '폐센터': '전문 센터 계열',
    '혈액종양내과': '내과 계열',
    '혈액종양내과 (암센터)': '내과 계열',
    '호흡기내과 (폐센터)': '내과 계열'
}
    df['department_group_bigggest'] = df['입원시 진료과'].replace(biggest_mapping)
    
    return df
    
def classify_department_bigger(df):
    bigger_mapping ={
    '감염내과': '기타 진료과',
    '관절센터': '정형외과·재활 계열',
    '내과': '내과 계열',
    '내분비대사내과': '내과 계열',
    '내분비대사내과 (노인의료센터)': '노인·종합진료',
    '노인병내과 (노인의료센터)': '노인·종합진료',
    '노인의료센터': '노인·종합진료',
    '뇌신경센터': '신경과·뇌 관련',
    '류마티스내과': '내과 계열',
    '마취통증의학과': '기타 진료과',
    '비뇨의학과': '외과 계열',
    '비뇨의학과 (암센터)': '암 치료 관련',
    '산부인과': '외과 계열',
    '성형외과': '외과 계열',
    '소아청소년과 (알레르기호흡기)': '기타 진료과',
    '소화기내과': '소화기·순환기 계열',
    '소화기내과 (암센터)': '암 치료 관련',
    '소화기센터': '소화기·순환기 계열',
    '순환기내과 (심장혈관센터)': '소화기·순환기 계열',
    '신경과 (뇌신경센터)': '신경과·뇌 관련',
    '신경외과': '외과 계열',
    '신경외과 (뇌신경센터)': '신경과·뇌 관련',
    '신경외과 (척추센터)': '정형외과·재활 계열',
    '신장내과': '내과 계열',
    '심장혈관센터': '소화기·순환기 계열',
    '심장혈관흉부외과 (심장혈관센터)': '소화기·순환기 계열',
    '심장혈관흉부외과 (폐센터)': '소화기·순환기 계열',
    '안과': '기타 진료과',
    '알레르기내과': '내과 계열',
    '외과': '외과 계열',
    '외과 (암센터)': '암 치료 관련',
    '응급의학과': '기타 진료과',
    '이비인후과': '기타 진료과',
    '재활의학과': '정형외과·재활 계열',
    '정형외과 (관절센터)': '정형외과·재활 계열',
    '정형외과 (암센터)': '암 치료 관련',
    '정형외과 (척추센터)': '정형외과·재활 계열',
    '종합내과': '노인·종합진료',
    '척추센터': '정형외과·재활 계열',
    '치과': '외과 계열',
    '폐센터': '소화기·순환기 계열',
    '혈액종양내과': '내과 계열',
    '혈액종양내과 (암센터)': '암 치료 관련',
    '호흡기내과 (폐센터)': '내과 계열'
}
    df['department_group_bigger'] = df['입원시 진료과'].replace(bigger_mapping)
    
    return df

In [75]:
공정성_관련_칼럼 = ['성별', '종교', '수진일 당시 직업', '수진 당시 나이', '수진일 당시 결혼상태', '입원시 진료과']

# 연령대 컬럼 추가(수진 당시 나이+1(만나이x))
test_df = 만나이플러스1(test_df)
valid_df = 만나이플러스1(valid_df)
공정성_관련_칼럼.append('수진 당시 나이+1(만나이x)')


test_df['연령대'] = test_df['수진 당시 나이+1(만나이x)'].apply(map_age_to_decade)
valid_df['연령대'] = valid_df['수진 당시 나이+1(만나이x)'].apply(map_age_to_decade)
공정성_관련_칼럼.append('연령대')


# 새로운 열 'age_group_big' 추가
test_df['age_group_big'] = test_df['수진 당시 나이+1(만나이x)'].apply(classify_age_group)
valid_df['age_group_big'] =  valid_df['수진 당시 나이+1(만나이x)'].apply(classify_age_group)
공정성_관련_칼럼.append('age_group_big')

# department
test_df = classify_department_biggest(test_df)
valid_df = classify_department_biggest(valid_df)
공정성_관련_칼럼.append('department_group_bigggest')

test_df = classify_department_bigger(test_df)
valid_df = classify_department_bigger(valid_df)
공정성_관련_칼럼.append('department_group_bigger')

/tmp/ipykernel_40820/1026885625.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['수진 당시 나이+1(만나이x)'] = data['수진 당시 나이']+1
/tmp/ipykernel_40820/3009707185.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df['연령대'] = valid_df['수진 당시 나이+1(만나이x)'].apply(map_age_to_decade)
/tmp/ipykernel_40820/3009707185.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [77]:
print(공정성_관련_칼럼)# ['성별', '종교', '수진일 당시 직업', '수진 당시 나이', '수진일 당시 결혼상태', '입원시 진료과', '수진 당시 나이+1(만나이x)', '연령대', 'age_group_big', 'department_group_bigggest', 'department_group_bigger']
공정성_관련_칼럼 = ['성별', '종교', '수진일 당시 직업', 
             '수진 당시 나이','수진 당시 나이+1(만나이x)', '연령대', 'age_group_big',
             '수진일 당시 결혼상태',
             '입원시 진료과', 'department_group_bigger' ,'department_group_bigggest']

for i in 공정성_관련_칼럼:
    print('\n')
    print(test_df[i].value_counts().sort_index())
    print('\n')

['성별', '종교', '수진일 당시 직업', '수진 당시 나이', '수진일 당시 결혼상태', '입원시 진료과', '수진 당시 나이+1(만나이x)', '연령대', 'age_group_big', 'department_group_bigggest', 'department_group_bigger']


성별
0    10201
1     7505
Name: count, dtype: int64




종교
0       243
개신교    4575
기타      294
무교     7655
불교     3013
천주교    1926
Name: count, dtype: int64




수진일 당시 직업
0       275
군인       92
기타     1479
무직     7451
자유업    1800
전문직     848
주부     3652
학생      194
회사원    1915
Name: count, dtype: int64




수진 당시 나이
19.0     42
20.0     55
21.0     38
22.0     65
23.0     34
       ... 
86.0    308
87.0    249
88.0    257
89.0    188
90.0    584
Name: count, Length: 72, dtype: int64




수진 당시 나이+1(만나이x)
20.0     42
21.0     55
22.0     38
23.0     65
24.0     34
       ... 
87.0    308
88.0    249
89.0    257
90.0    188
91.0    584
Name: count, Length: 72, dtype: int64




연령대
20.0대     409
30.0대     701
40.0대    1216
50.0대    2256
60.0대    3882
70.0대    4594
80.0대    3876
90.0대     772
Name: count, dtype: int64




age_gr

In [78]:
# 공정성_관련_칼럼 = ['성별', 
#                    '종교', 
#                    '수진일 당시 직업', 
#                    '수진 당시 나이','수진 당시 나이+1(만나이x)', '연령대', 'age_group_big',
#                    '수진일 당시 결혼상태',
#                    '입원시 진료과', 'department_group_bigger' ,'department_group_bigggest']

y_pred = model_xgb_pre.predict(X_test)  # 이진 예측
y_proba = model_xgb_pre.predict_proba(X_test)[:, 1]  # 클래스 1의 확률 예측
y_test # 실제 레이블

s_f_성별 = test_df['성별']
s_f_종교 = test_df['종교']
s_f_직업 = test_df['수진일 당시 직업']
s_f_연령대 = test_df['연령대']
s_f_결혼상태 = test_df['수진일 당시 결혼상태']
s_f_department = test_df['department_group_bigger']
             
data = {
    'y_pred': y_pred,
    'y_proba': y_proba,
    'y_true': y_test,
    '성별': s_f_성별,
    '종교': s_f_종교,
    '수진일 당시 직업': s_f_직업,
    '연령대' : s_f_연령대,
    '수진일 당시 결혼상태' : s_f_결혼상태,
    'department_group_bigger' : s_f_department
}
res = pd.DataFrame(data)


In [79]:
# Disparate Impact 계산 함수
def calculate_disparate_impact(df, attribute, positive_outcome=True):
    # 긍정적 결과를 받은 비율 계산
    if positive_outcome:
        favorable_rate = df.groupby(attribute)['y_pred'].mean()
    else:
        favorable_rate = 1 - df.groupby(attribute)['y_pred'].mean()

    # 보호된 속성에 따른 비율 계산
    di_values = favorable_rate / favorable_rate.max()
    return di_values

# Equality of Opportunity Difference 계산 함수
def calculate_eod(df, attribute, positive_class=1):
    # 실제 긍정 클래스인 경우에 한정
    positive_actual = df[df['y_true'] == positive_class]

    # 긍정적 결과를 받은 비율 계산
    opportunity_rate = positive_actual.groupby(attribute)['y_pred'].mean()
    
    # 최소 및 최대 그룹 간의 차이
    eod_value = opportunity_rate.max() - opportunity_rate.min()
    return eod_value

In [84]:
a = 'di_성별'
a[-2:]

'성별'

In [94]:
res.columns[3:]

Index(['성별', '종교', '수진일 당시 직업', '연령대', '수진일 당시 결혼상태',
       'department_group_bigger'],
      dtype='object')

In [96]:
print(res.columns)

# 예시로 'race_group'에 대한 DI와 EOD 계산
di_성별 = calculate_disparate_impact(res, '성별')
eod_성별 = calculate_eod(res, '성별')

di_종교 = calculate_disparate_impact(res, '종교')
eod_종교 = calculate_eod(res, '종교')

di_직업 = calculate_disparate_impact(res, '수진일 당시 직업')
eod_직업 = calculate_eod(res, '수진일 당시 직업')

di_age = calculate_disparate_impact(res, '연령대')
eod_age = calculate_eod(res, '연령대')

di_결혼 = calculate_disparate_impact(res, '수진일 당시 결혼상태')
eod_결혼 = calculate_eod(res, '수진일 당시 결혼상태')

di_department = calculate_disparate_impact(res, 'department_group_bigger')
eod_department = calculate_eod(res, 'department_group_bigger')


def print_eod_di(di,eod,i):
    b = res.columns[3:][i]
    print('\n\n-------------------------------------------------')
    
    print(f"Disparate Impact(DI) by {b}:")
    print(di)
    print('\n')
    print(f"\n\nEquality of Opportunity Difference(EOD) by {b}:")
    print(eod)
    # print('-------------------------------------------------\n\n\n\n')

print_eod_di(di_성별,eod_성별,0)
print_eod_di(di_종교,eod_종교,1)
print_eod_di(di_직업,eod_직업,2)
print_eod_di(di_age,eod_age,3)
print_eod_di(di_결혼,eod_결혼,4)
print_eod_di(di_department,eod_department,5)

Index(['y_pred', 'y_proba', 'y_true', '성별', '종교', '수진일 당시 직업', '연령대',
       '수진일 당시 결혼상태', 'department_group_bigger'],
      dtype='object')


-------------------------------------------------
Disparate Impact(DI) by 성별:
성별
0    1.000000
1    0.732588
Name: y_pred, dtype: float64




Equality of Opportunity Difference(EOD) by 성별:
0.015801132168348553


-------------------------------------------------
Disparate Impact(DI) by 종교:
종교
0      0.837120
개신교    0.949815
기타     0.956891
무교     0.980958
불교     0.909287
천주교    1.000000
Name: y_pred, dtype: float64




Equality of Opportunity Difference(EOD) by 종교:
0.08052775250227484


-------------------------------------------------
Disparate Impact(DI) by 수진일 당시 직업:
수진일 당시 직업
0      0.757889
군인     0.566358
기타     0.674064
무직     1.000000
자유업    0.600171
전문직    0.548904
주부     0.567846
학생     0.465542
회사원    0.642128
Name: y_pred, dtype: float64




Equality of Opportunity Difference(EOD) by 수진일 당시 직업:
0.24137931034482762


-----------------

- DI, EOD 공정성 metrics 평가
    - Disparate Impact (DI): 보호 속성에 따른 그룹에서 긍정 예측 비율의 최소값과 최대값의 비율로 계산합니다. 값이 1에 가까울수록 공정하다고 판단됩니다.
    - Equal Opportunity Differences (EOD): 실제 긍정인 경우, 보호 속성에 따른 그룹 간의 예측 긍정 비율 차이를 계산합니다. 값이 0에 가까울수록 공정하다고 판단됩니다.

# 공정성 점수 올리는 작업
# case6개 , stage3개 라벨링

In [171]:
y_pred = model_xgb_pre.predict(X_test)  # 이진 예측
y_proba = model_xgb_pre.predict_proba(X_test)[:, 1]  # 클래스 1의 확률 예측
# y_test # 실제 레이블
# len(y_pred)

print(f"보호변수 : {protect_variable}")
# s_f_age = test_dftest_df['age_group'] # 이게 아마 보호변수인듯?
s_f_age = test_df['age_group_big'] # 수진 당시 나이
s_f_gender = test_df['성별']
s_f_종교 = test_df['종교']
s_f_직업 = test_df['수진일 당시 직업']
s_f_결혼상태 = test_df['수진일 당시 결혼상태']
s_f_department = test_df['입원시 진료과']
print(len(s_f_age))
print('\n')
# # 예측 결과와 실제 레이블, 보호변수를 데이터프레임으로 구성
# data = {
#     'y_pred': y_pred,
#     'y_proba': y_proba,
#     'y_true': y_test,
#     'age_group': s_f_age,
#     'gender': s_f_gender,
#     'race_group': s_f_race
# }
# res = pd.DataFrame(data)

# # Disparate Impact 계산 함수
# def calculate_disparate_impact(df, attribute, positive_outcome=True):
#     # 긍정적 결과를 받은 비율 계산
#     if positive_outcome:
#         favorable_rate = df.groupby(attribute)['y_pred'].mean()
#     else:
#         favorable_rate = 1 - df.groupby(attribute)['y_pred'].mean()

#     # 보호된 속성에 따른 비율 계산
#     di_values = favorable_rate / favorable_rate.max()
#     return di_values

# # Equality of Opportunity Difference 계산 함수
# def calculate_eod(df, attribute, positive_class=1):
#     # 실제 긍정 클래스인 경우에 한정
#     positive_actual = df[df['y_true'] == positive_class]

#     # 긍정적 결과를 받은 비율 계산
#     opportunity_rate = positive_actual.groupby(attribute)['y_pred'].mean()
    
#     # 최소 및 최대 그룹 간의 차이
#     eod_value = opportunity_rate.max() - opportunity_rate.min()
#     return eod_value

# # 예시로 'race_group'에 대한 DI와 EOD 계산
# di_race = calculate_disparate_impact(res, 'race_group')
# eod_race = calculate_eod(res, 'race_group')

# di_age = calculate_disparate_impact(res, 'age_group')
# eod_age = calculate_eod(res, 'age_group')

# di_gender = calculate_disparate_impact(res, 'gender')
# eod_gender = calculate_eod(res, 'gender')


# print("Disparate Impact(DI) by Race:")
# print(di_race)
# print("\n\nEquality of Opportunity Difference(EOD) by Race:")
# print(eod_race)
# print('\n\n\n\n-------------------------------------------------')
# print("Disparate Impact(DI) by age:")
# print(di_age)
# print("\n\nEquality of Opportunity Difference(EOD) by age:")
# print(eod_age)
# print('\n\n\n\n-------------------------------------------------')
# print("Disparate Impact(DI) by gender:")
# print(di_gender)
# print("\n\nEquality of Opportunity Difference(EOD) by gender:")
# print(eod_gender)


보호변수 : ['성별', '종교', '수진일 당시 직업', '수진 당시 나이', '수진일 당시 결혼상태', '입원시 진료과']
